In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, f1_score

# -----------------------------------------------------------------------------
# Utility function (giống code gốc PhysioNet)
# -----------------------------------------------------------------------------
def compute_prediction_utility_single(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3,
                                      max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0):
    labels = np.array(labels, dtype=int)
    predictions = np.array(predictions, dtype=int)

    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)
    m_1 = max_u_tp / (dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = -max_u_tp / (dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = min_u_fn / (dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    utility = 0.0
    for t in range(n):
        if t > t_sepsis + dt_late:
            if not is_septic and predictions[t] == 1:
                utility += u_fp
            continue

        if is_septic and predictions[t] == 1:  # TP
            if t <= t_sepsis + dt_early:
                u_val = u_fp
            elif t <= t_sepsis + dt_optimal:
                u_val = m_1 * (t - t_sepsis) + b_1
            else:
                u_val = m_2 * (t - t_sepsis) + b_2
            utility += max(u_val, u_fp)

        elif not is_septic and predictions[t] == 1:  # FP
            utility += u_fp

        elif is_septic and predictions[t] == 0:  # FN
            if t <= t_sepsis + dt_optimal:
                u_val = 0
            else:
                u_val = m_3 * (t - t_sepsis) + b_3
            utility += u_val

        else:  # TN
            utility += u_tn

    return utility


# -----------------------------------------------------------------------------
# Load raw probabilities (không binarize trước)
# -----------------------------------------------------------------------------
def load_probabilities_and_labels(folder):
    all_probs = []
    all_trues = []
    filenames = []

    files = sorted([f for f in os.listdir(folder) if f.endswith(".psv")])
    for f in files:
        df = pd.read_csv(os.path.join(folder, f), sep="|")
        all_probs.append(df["PredictedProbability"].values.astype(float))
        all_trues.append(df["TrueLabel"].values.astype(int))
        filenames.append(f)
    return all_probs, all_trues


# -----------------------------------------------------------------------------
# Tính Normalized Utility cho một ngưỡng cụ thể
# -----------------------------------------------------------------------------
def compute_normalized_utility_at_threshold(all_probs, all_trues, threshold):
    all_preds = [ (probs >= threshold).astype(int) for probs in all_probs ]

    total_utility = 0.0
    best_utility = 0.0
    inaction_utility = 0.0

    for labels, preds in zip(all_trues, all_preds):
        labels = np.array(labels, dtype=int)
        preds = np.array(preds, dtype=int)

        total_utility += compute_prediction_utility_single(labels, preds)

        # Best (oracle) predictions
        num_rows = len(labels)
        best_preds = np.zeros(num_rows, dtype=int)
        if np.any(labels):
            t_sepsis = np.argmax(labels) - (-6)  # dt_optimal = -6
            start = max(0, int(t_sepsis - 12))
            end = min(num_rows, int(t_sepsis + 3 + 1))
            best_preds[start:end] = 1
        best_utility += compute_prediction_utility_single(labels, best_preds)

        # Inaction (all zero)
        inaction_utility += compute_prediction_utility_single(labels, np.zeros_like(labels))

    if (best_utility - inaction_utility) != 0:
        norm_utility = (total_utility - inaction_utility) / (best_utility - inaction_utility)
    else:
        norm_utility = 0.0

    return norm_utility



# -----------------------------------------------------------------------------
# Tìm ngưỡng tối ưu + LOG tiến trình
# -----------------------------------------------------------------------------
def find_optimal_threshold(folder, thresholds=np.arange(0.01, 0.51, 0.01), verbose=True):
    all_probs, all_trues = load_probabilities_and_labels(folder)

    flat_probs = np.concatenate(all_probs)
    flat_trues = np.concatenate(all_trues)

    # Global metrics (AUROC/AUPRC)
    if len(np.unique(flat_trues)) < 2:
        AUROC = np.nan
        AUPRC = np.nan
    else:
        AUROC = roc_auc_score(flat_trues, flat_probs)
        AUPRC = average_precision_score(flat_trues, flat_probs)

    results = []
    best_utility = -np.inf
    best_threshold = thresholds[0]

    if verbose:
        print(f"🔍 Đang tìm ngưỡng tối ưu trong khoảng [{thresholds[0]:.3f}, {thresholds[-1]:.3f}] (bước = {thresholds[1]-thresholds[0]:.3f})")
        print("-" * 60)

    for i, th in enumerate(thresholds):
        norm_util = compute_normalized_utility_at_threshold(all_probs, all_trues, th)
        flat_preds = (flat_probs >= th).astype(int)
        acc = accuracy_score(flat_trues, flat_preds)
        f1 = f1_score(flat_trues, flat_preds, zero_division=0)

        results.append({
            "Threshold": th,
            "Normalized_Utility": norm_util,
            "Accuracy": acc,
            "F1": f1
        })

        if norm_util > best_utility:
            best_utility = norm_util
            best_threshold = th

        if verbose:
            print(f"[{i+1:2d}/{len(thresholds)}] Threshold = {th:.3f} → Utility = {norm_util:.5f} (Best so far: {best_utility:.5f} @ {best_threshold:.3f})")

    if verbose:
        print("-" * 60)
        print(f"✅ Optimal Threshold: {best_threshold:.3f}")
        print(f"   Normalized Utility: {best_utility:.5f}")
        print(f"   AUROC: {AUROC:.5f}, AUPRC: {AUPRC:.5f}")

    return best_threshold, pd.DataFrame(results)


# -----------------------------------------------------------------------------
# CHẠY
# -----------------------------------------------------------------------------
folder_path = "/kaggle/input/file-optimal-threshold-on-old-model"
optimal_th, all_results = find_optimal_threshold(folder_path, thresholds=np.arange(0.01, 0.91, 0.01))

🔍 Đang tìm ngưỡng tối ưu trong khoảng [0.010, 0.900] (bước = 0.010)
------------------------------------------------------------
[ 1/90] Threshold = 0.010 → Utility = -0.31698 (Best so far: -0.31698 @ 0.010)
[ 2/90] Threshold = 0.020 → Utility = -0.31696 (Best so far: -0.31696 @ 0.020)
[ 3/90] Threshold = 0.030 → Utility = -0.31667 (Best so far: -0.31667 @ 0.030)
[ 4/90] Threshold = 0.040 → Utility = -0.31559 (Best so far: -0.31559 @ 0.040)
[ 5/90] Threshold = 0.050 → Utility = -0.31307 (Best so far: -0.31307 @ 0.050)
[ 6/90] Threshold = 0.060 → Utility = -0.30865 (Best so far: -0.30865 @ 0.060)
[ 7/90] Threshold = 0.070 → Utility = -0.30193 (Best so far: -0.30193 @ 0.070)
[ 8/90] Threshold = 0.080 → Utility = -0.29403 (Best so far: -0.29403 @ 0.080)
[ 9/90] Threshold = 0.090 → Utility = -0.28519 (Best so far: -0.28519 @ 0.090)
[10/90] Threshold = 0.100 → Utility = -0.27543 (Best so far: -0.27543 @ 0.100)
[11/90] Threshold = 0.110 → Utility = -0.26543 (Best so far: -0.26543 @ 0.110)
[1